## Workshop 1: Data Manipulation with Pandas & NumPy

### Advanced Indexing, Filtering, and Handling Missing Data

------

## 1. Scenario Overview

You are a **Data Engineer at "UrbanHealth Analytics"**, a healthcare startup that aggregates patient visit data from multiple clinics across West Africa. Your data pipeline receives daily exports, but the data quality is poor:

- Missing vital signs (blood pressure, temperature)
- Duplicate patient records with different IDs
- Inconsistent date formats and timezones
- Mixed units (kg vs lbs, Celsius vs Fahrenheit)
- Incomplete clinic location data
- Invalid age values (negative numbers, >150 years)

**Business Impact:** Poor data quality has caused:

- 15% of patients to be double-billed
- Inaccurate health trend reports for government partners
- Failed ML model predictions for disease outbreak detection

**Your Mission:** Build a **production-grade data cleaning pipeline** that processes 100K+ records daily, with proper error handling, logging, and data validation.

------

## 2. Learning Outcomes

By the end of this workshop, you will:

- **Master efficient data manipulation** using vectorized operations 
- **Understand when and why** to use different indexing methods (`.loc`, `.iloc`, multi-index) 
- **Handle missing data strategically** based on business context 
- **Identify performance bottlenecks** in data pipelines 
- **Write production-ready, modular code** with proper error handling 
- **Make data-driven decisions** about cleaning strategies 
- **Document trade-offs** between different approaches

------

## 3. Architecture & Data Flow

```
┌─────────────────┐
│ Clinic Systems  │ (CSV/API exports - Multiple sources)
│ - Clinic A      │
│ - Clinic B      │
│ - Clinic C      │
└────────┬────────┘
         │
         ▼
┌─────────────────────────┐
│ RAW DATA LAYER          │
│ - S3/Cloud Storage      │
│ - Daily partitions      │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│ DATA CLEANING PIPELINE  │ ← WE ARE HERE
│ ┌─────────────────────┐ │
│ │ 1. Validation       │ │
│ │ 2. Deduplication    │ │
│ │ 3. Missing Handling │ │
│ │ 4. Standardization  │ │
│ │ 5. Feature Eng      │ │
│ └─────────────────────┘ │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│ CLEAN DATA LAYER        │
│ - Parquet files         │
│ - Quality metrics       │
└────────┬────────────────┘
         │
         ├──────────┬─────────────┐
         ▼          ▼             ▼
   ┌─────────┐ ┌────────┐  ┌──────────┐
   │Analytics│ │ML Model│  │BI Reports│
   └─────────┘ └────────┘  └──────────┘
```

------

## 4. The Dataset: Understanding Your Data

### 4.1 Initial Exploration 

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Load sample data
df = pd.DataFrame({
    "visit_id": [1001, 1002, 1003, 1004, 1005, 1005, 1006, 1007],
    "patient_id": ["P001", "P002", "P003", "P004", np.nan, "P004", "P005", "P006"],
    "age": [34, 150, 28, -5, 45, 45, 62, 29],
    "blood_pressure_sys": [120, 140, np.nan, 110, 180, 180, np.nan, 125],
    "blood_pressure_dia": [80, 90, np.nan, 70, 95, 95, 85, np.nan],
    "temperature_c": [36.5, 37.2, 38.9, np.nan, 36.8, 36.8, 37.1, 98.6],  # Mixed units!
    "visit_date": ["2024-01-15", "2024-01-15", "2024-01-16", "2024-01-16", 
                   "2024-01-17", "2024-01-17", "2024-01-18", "not_a_date"],
    "clinic_id": ["C001", "C001", "C002", np.nan, "C002", "C002", "C003", "C001"],
    "weight_kg": [68.5, 72.0, np.nan, 85.2, 90.0, 90.0, 165.3, 58.0]  # 165.3 lbs not kg!
})

